In [196]:
import pandas as pd
import os  

In [197]:
all_data = []

# zum Verzeichnis navigieren

for year in range(1960, 2025):
    data_path = os.path.join('..', 'Data', f'{year}', f'merged_movies_data_{year}.csv')

    if os.path.exists(data_path):  # Prüfen, ob die Datei existiert
        df = pd.read_csv(data_path)
        all_data.append(df)

    else:
        print(f'Datei fuer {year} nicht gefunden: {data_path}')

# Alle df's zusammenfuegen
final_df = pd.concat(all_data, ignore_index = True)

In [198]:
# Neue Spalte mit der ID

def get_id(link):
    part = link.split("title/")[1].split("/")[0]
    return part

final_df['id'] = final_df['Movie Link'].apply(get_id)

In [199]:
def duration_to_minutes(duration):
    
    if pd.isna(duration):
        return 0


    # Initialisiere Stunden und Minuten
    hours, minutes = 0, 0

    # Wenn 'h' im String ist, extrahiere die Stunden
    if 'h' in duration:
        hours = int(duration.split('h')[0].strip())

    # Wenn 'm' im String ist, extrahiere die Minuten
    if 'm' in duration:
        # Split an 'h' und nimm den rechten Teil, falls vorhanden
        minutes_part = duration.split('h')[-1]
        minutes = int(minutes_part.replace('m', '').strip())

    return hours * 60 + minutes

# Wende die Funktion auf die 'Duration'-Spalte an
final_df['Duration'] = final_df['Duration'].apply(duration_to_minutes)

In [200]:
# Votes korrigieren
def correct_votes(votes):
    if pd.isna(votes):
        return votes

    if 'K' in votes:
        # Entferne 'K' und multipliziere den Wert mit 1000
        votes_correct = float(votes.replace('K', '')) * 1000
    # Falls 'M' im Wert enthalten ist
    elif 'M' in votes:
        # Entferne 'M' und multipliziere den Wert mit 1000000
        votes_correct = float(votes.replace('M', '')) * 1000000
    else:
        # Falls keine der beiden Einheiten vorhanden ist, wird der Wert als Ganzzahl übernommen
        votes_correct = int(votes)
    
    return int(votes_correct)

final_df['Votes'] = final_df['Votes'].apply(correct_votes)

In [201]:
# 3. Listen-Spalten sauber formatieren
def list_columns_format(df):
    
    list_columns = ['writers', 'stars', 'genres', 'countries_origin',
                    'filming_locations', 'production_companies']
    for col in list_columns:
        df[col] = df[col].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    return df

list_columns_format(df)

,Title,Movie Link,Year,Duration,MPA,Rating,Votes,Description,budget,grossWorldWide,...,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars,release_date
0,1. Inside Out 2,https://www.imdb.com/title/tt22022452/?ref_=sr...,2024,1h 36m,PG,7.5,204K,A sequel that features Riley entering puberty ...,200000000.0,1698863816,...,"[Amy Poehler, Maya Hawke, Kensington Tallman]","[Coming-of-Age, Computer Animation, Quest, Tee...",[United States],[Walt Disney Feature Animation - 500 S. Buena ...,"[Walt Disney Pictures, Pixar Animation Studios...","['English', 'French', 'Spanish']",3,90,1,2024
1,2. Deadpool & Wolverine,https://www.imdb.com/title/tt6263850/?ref_=sr_t_2,2024,2h 8m,R,7.6,466K,Deadpool is offered a place in the Marvel Cine...,200000000.0,1338073645,...,"[Ryan Reynolds, Hugh Jackman, Emma Corrin]","[Buddy Comedy, Dark Comedy, Raunchy Comedy, Su...","[United States, United Kingdom, New Zealand, C...","[Vancouver Film Studios - 3500 Cornett Rd, Van...","[Marvel Studios, 20th Century Studios, Maximum...","['English', 'French', 'Spanish']",13,44,0,2024
2,3. Wicked,https://www.imdb.com/title/tt1262426/?ref_=sr_t_3,2024,2h 40m,PG,7.6,125K,"Elphaba, a misunderstood young woman because o...",150000000.0,723338105,...,"[Cynthia Erivo, Ariana Grande, Jeff Goldblum]","[Fairy Tale, Pop Musical, Fantasy, Musical, Ro...","[United States, Japan, Canada, Iceland, United...","[Sky Studios Elstree Rowley, Borehamwood, Hert...","[Universal Pictures, Marc Platt Productions, M...",['English'],70,264,10,2024
3,4. Moana 2,https://www.imdb.com/title/tt13622970/?ref_=sr...,2024,1h 40m,PG,6.8,76K,After receiving an unexpected call from her wa...,150000000.0,1044203767,...,"[Auli'i Cravalho, Dwayne Johnson, Hualalai Chung]","[Computer Animation, Sea Adventure, Adventure,...","[United States, Canada]","[Walt Disney Animation Studios, Vancouver BC]","[Disney Television Animation, The Walt Disney ...",['English'],2,23,0,2024
4,5. Despicable Me 4,https://www.imdb.com/title/tt7510222/?ref_=sr_t_5,2024,1h 34m,PG,6.2,59K,"Gru, Lucy, Margo, Edith, and Agnes welcome a n...",100000000.0,969126452,...,"[Steve Carell, Kristen Wiig, Pierre Coffin]","[Computer Animation, Urban Adventure, Adventur...",[United States],[],"[Universal Pictures, Illumination Entertainment]","['English', 'French']",0,3,0,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96. Cuckoo,https://www.imdb.com/title/tt12349832/?ref_=sr...,2024,1h 42m,R,5.7,23K,A 17-year-old girl is forced to move with her ...,7000000.0,6680396,...,"[Hunter Schafer, Jan Bluthardt, Marton Csokas]","[Teen Horror, Horror, Mystery, Thriller]","[Germany, United States]","[Wuppertal, North Rhine-Westphalia, Germany]","[Fiction Park, Neon, Waypoint Entertainment]","['English', 'German', 'French', 'American Sign...",2,11,0,2024
96,97. Back to Black,https://www.imdb.com/title/tt21261712/?ref_=sr...,2024,2h 2m,R,6.3,25K,"The life and music of Amy Winehouse, through t...",30000000.0,51026615,...,"[Marisa Abela, Eddie Marsan, Jack O'Connell]","[Docudrama, Biography, Drama, Music]","[United Kingdom, France, United States]","[Good Mixer, 30 Inverness Street, London, NW1 ...","[StudioCanal UK, Monumental Pictures, Canal+]",['English'],0,7,0,2024
97,98. Someone Like You,https://www.imdb.com/title/tt22334258/?ref_=sr...,2024,1h 58m,PG,5.7,1.8K,"After the tragic loss of his best friend, a gr...",NaN,5937306,...,"[Sarah Fisher, Jake Allyn, Lynn Collins]",[Romance],[United States],"[Nashville, Tennessee, USA]",[Karen Kingsbury Productions],['English'],0,0,0,2024
98,99. Devara Part 1,https://www.imdb.com/title/tt11821912/?ref_=sr...,2024,2h 56m,NaN,6.0,20K,A mighty sea warrior takes a violent stand aga...,36500000.0,7361414,...,"[Saif Ali Khan, N.T. Rama Rao Jr., Janhvi Kapoor]","[Action Epic, Epic, One-Person Army Action, Se...",[India],"[Goa, India]","[NTR Arts, Yuvasudha Arts]",['Telugu'],3,0,0,2024


In [202]:
final_df.head(3)

,Title,Movie Link,Year,Duration,MPA,Rating,Votes,Description,budget,grossWorldWide,...,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars,release_date,id
0,1. Swiss Family Robinson,https://www.imdb.com/title/tt0054357/?ref_=sr_t_1,1960,126,Approved,7.1,19000.0,A Swiss family must survive being shipwrecked ...,5000000.0,40357287.0,...,"['Survival', 'Adventure', 'Family']",['United States'],"['Tobago, Trinidad and Tobago']",['Walt Disney Productions'],"['English', 'Malay']",0,0,0,1960,tt0054357
1,2. Psycho,https://www.imdb.com/title/tt0054215/?ref_=sr_t_2,1960,109,R,8.5,745000.0,A secretary on the run for embezzlement takes ...,806947.0,32248065.0,...,"['Dark Comedy', 'Psychological Horror', 'Psych...",['United States'],"['Psycho House and Bates Motel, Backlot Univer...","['Alfred J. Hitchcock Productions', 'Shamley P...",['English'],8,14,4,1960,tt0054215
2,3. The Apartment,https://www.imdb.com/title/tt0053604/?ref_=sr_t_3,1960,125,Approved,8.3,206000.0,A Manhattan insurance clerk tries to rise in h...,3000000.0,18783481.0,...,"['Farce', 'Holiday Comedy', 'Holiday Romance',...",['United States'],"['Majestic Theater, 247 West 44th Street, Manh...",['The Mirisch Corporation'],['English'],25,8,5,1960,tt0053604


In [203]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6600 entries, 0 to 6599
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  6600 non-null   object 
 1   Movie Link             6600 non-null   object 
 2   Year                   6600 non-null   int64  
 3   Duration               6600 non-null   int64  
 4   MPA                    6447 non-null   object 
 5   Rating                 6598 non-null   float64
 6   Votes                  6598 non-null   float64
 7   Description            6599 non-null   object 
 8   budget                 4981 non-null   float64
 9   grossWorldWide         5378 non-null   float64
 10  gross_US_Canada        5253 non-null   float64
 11  opening_weekend_Gross  4659 non-null   float64
 12  directors              6600 non-null   object 
 13  writers                6600 non-null   object 
 14  stars                  6600 non-null   object 
 15  genr

In [204]:
# Es fehlen nur zwei Werte da, deshalb fallen die Filme raus.
final_df = final_df.dropna(subset=['Votes'])

In [205]:
# Mapping der MPA-Kategorien
mpa_categories = {
    "All Ages (+0)": ['G', 'TV-G', 'TV-Y', 'TV-Y7', 'K-A', 'Approved'],
    "Parental Guidance (+13)": ['PG', 'PG-13', 'M/PG', 'TV-PG', 'TV-13', 'TV-14'],
    "Mature Audiences (+18)": ['R', 'MA-17', 'TV-MA', 'NC-17', 'X'],
    "Not Rated/Other": ['Not Rated', 'Unrated', 'Passed', 'GP']
}

# Mapping umkehren für schnelles Zuordnen
mpa_mapping = {}
for category, labels in mpa_categories.items():
    for label in labels:
        mpa_mapping[label] = category

# Spalte in allgemeine Kategorien umwandeln
final_df['MPA_category'] = final_df['MPA'].map(mpa_mapping)

# Fehlende Werte in `MPA_category` mit "Unknown" auffüllen
final_df['MPA_category'].fillna("Unknown", inplace=True)

# Alte `MPA`-Spalte entfernen
final_df.drop(columns=['MPA'], inplace=True)

C:\Users\diver\AppData\Local\Temp\ipykernel_25416\4031684665.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['MPA_category'].fillna("Unknown", inplace=True)


In [206]:
# Zählen von fehlenden Werten (NaNs) und leeren Strings in jeder Spalte
missing_values = final_df.isnull().sum()  # Zählt NaNs
empty_strings = (final_df == "").sum()   # Zählt leere Strings

# Kombiniere die Ergebnisse in einem DataFrame zur besseren Übersicht
missing_summary = pd.DataFrame({'NaNs': missing_values, 'Empty Strings': empty_strings})
missing_summary['Total Missing'] = missing_summary['NaNs'] + missing_summary['Empty Strings']

# Zeige die Spalten an, die mindestens eine fehlende oder leere Zelle haben
missing_summary = missing_summary[missing_summary['Total Missing'] > 0]

missing_summary

,NaNs,Empty Strings,Total Missing
budget,1617,0,1617
grossWorldWide,1221,0,1221
gross_US_Canada,1346,0,1346
opening_weekend_Gross,1939,0,1939


In [207]:
# Angenommene Spalten, die die NaN-Werte enthalten
columns_to_replace = ['budget', 'grossWorldWide', 'gross_US_Canada', 'opening_weekend_Gross']

# Funktion zum Ersetzen der NaN-Werte mit dem Median pro Jahr
def replace_na_with_median(df, columns):
    for column in columns:
        # Group by 'Year' und berechne den Median pro Jahr
        df[column] = df.groupby('Year')[column].transform(lambda x: x.fillna(x.median()))
    
    return df

# Anwenden der Funktion auf das DataFrame
final_df = replace_na_with_median(final_df, columns_to_replace)

In [208]:
missing_values_per_year = final_df.isna().groupby(final_df['Year']).sum()
missing_values_per_year

,Title,Movie Link,Year,Duration,Rating,Votes,Description,budget,grossWorldWide,gross_US_Canada,...,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars,release_date,id,MPA_category
Year,,,,,,,,,,,,,,,,,,,,,
1960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1963,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
# Neue Spalte fuer inflationsbereinigtes opening_weekend_gross
def inflation(year, gross, base_year):

    cpi_data = {
        1960: 29.55, 1961: 29.9, 1962: 30.15, 1963: 30.65, 1964: 31.05, 
        1965: 31.45, 1966: 32.2, 1967: 33.15, 1968: 34.0, 1969: 35.05, 
        1970: 38.0, 1971: 40.2, 1972: 41.4, 1973: 43.3, 1974: 46.2, 
        1975: 52.3, 1976: 56.4, 1977: 60.4, 1978: 63.4, 1979: 69.2, 
        1980: 80.6, 1981: 90.2, 1982: 94.4, 1983: 99.1, 1984: 102.5, 
        1985: 106.2, 1986: 109.4, 1987: 112.6, 1988: 116.0, 1989: 121.5, 
        1990: 130.7, 1991: 135.7, 1992: 139.7, 1993: 143.2, 1994: 146.0, 
        1995: 151.6, 1996: 154.6, 1997: 160.0, 1998: 162.0, 1999: 165.0, 
        2000: 171.3, 2001: 175.8, 2002: 179.9, 2003: 183.5, 2004: 188.2, 
        2005: 192.8, 2006: 199.1, 2007: 203.2, 2008: 214.0, 2009: 215.1, 
        2010: 218.0, 2011: 221.6, 2012: 229.1, 2013: 232.8, 2014: 235.4, 
        2015: 236.8, 2016: 238.5, 2017: 243.2, 2018: 249.0, 2019: 254.4, 
        2020: 259.0, 2021: 267.8, 2022: 286.2, 2023: 303.5, 2024: 312.0  # 2024 als Referenz
    }

    return (gross / cpi_data[year]) * cpi_data[base_year]


final_df['inflation_gross_opening'] = final_df.apply(lambda row: inflation(row['Year'], row['opening_weekend_Gross'], 2024), axis=1)
final_df['budget_inflation'] = final_df.apply(lambda row: inflation(row['Year'], row['budget'], 2024), axis=1)

In [210]:
def has_oscars(x):
    if x == 0:
        return 0
    else:
        return 1

final_df['has_oscars'] = final_df['oscars'].apply(has_oscars)

In [211]:
def assign_decade(year):

    decades = {'60s': range(1960, 1970),
               '70s': range(1970, 1980),
               '80s': range(1980, 1990),
               '90s': range(1990, 2000),
               '00s': range(2000, 2010),
               '2010s': range(2010, 2020),
               '2020s': range(2020, 2030)}

    for decade, years in decades.items():
        if year in years:
            return decade

    return 'Unknown'


# Neue Spalte fuer Jahrzente
final_df['decade'] = final_df['Year'].apply(assign_decade)

In [212]:
# Einzigartige Subgenres extrahieren
import ast

# Liste, um einzigartige Subgenres zu speichern
unique_subgenres = []

# Iteration über die 'genres'-Spalte
for genre_list in final_df['genres']:
    # Umwandeln des Strings in eine Liste
    genre_list = ast.literal_eval(genre_list)
    
    # Hinzufügen der Subgenres zur Liste, wenn sie noch nicht vorhanden sind
    for subgenre in genre_list:
        if subgenre not in unique_subgenres:
            unique_subgenres.append(subgenre)

unique_subgenres

['Survival',
 'Adventure',
 'Family',
 'Dark Comedy',
 'Psychological Horror',
 'Psychological Thriller',
 'Slasher Horror',
 'Suspense Mystery',
 'Horror',
 'Mystery',
 'Thriller',
 'Farce',
 'Holiday Comedy',
 'Holiday Romance',
 'Romantic Comedy',
 'Comedy',
 'Drama',
 'Holiday',
 'Romance',
 'Adventure Epic',
 'Epic',
 'Historical Epic',
 'Sword & Sandal',
 'War Epic',
 'Biography',
 'History',
 'War',
 'Caper',
 'Crime',
 'Period Drama',
 'Tragic Romance',
 'Body Horror',
 'Tragedy',
 'Classical Western',
 'Action',
 'Western',
 'Music',
 'Dystopian Sci-Fi',
 'Quest',
 'Steampunk',
 'Time Travel',
 'Sci-Fi',
 'Legal Drama',
 'Space Sci-Fi',
 'Monster Horror',
 'Desert Adventure',
 'Western Epic',
 'Sport',
 'Supernatural Horror',
 'Vampire Horror',
 'Witch Horror',
 'Dinosaur Adventure',
 'Jungle Adventure',
 'Fantasy',
 'Feel-Good Romance',
 'Slapstick',
 'True Crime',
 'Musical',
 'Jukebox Musical',
 'Rock Musical',
 'Screwball Comedy',
 'Parody',
 'Satire',
 'Quirky Comedy',
 '

In [213]:
genre_categories = {
    "Action": [
        'Action', 'Action Epic', 'B-Action', 'Car Action', 'Gun Fu', 
        'Martial Arts', 'One-Person Army Action', 'Kung Fu', 
        'Samurai', 'Sword & Sandal', 'Swashbuckler', 'Buddy Cop', 
        'Spy'
    ],
    "Abenteuer": [
        'Adventure', 'Adventure Epic', 'Animal Adventure', 'Desert Adventure', 
        'Dinosaur Adventure', 'Globetrotting Adventure', 'Jungle Adventure', 
        'Mountain Adventure', 'Sea Adventure', 'Space Sci-Fi', 'Urban Adventure', 
        'Quest', 'Survival', 'Road Trip'
    ],
    "Horror": [
        'Horror', 'B-Horror', 'Monster Horror', 'Psychological Horror', 
        'Slasher Horror', 'Supernatural Horror', 'Teen Horror', 
        'Vampire Horror', 'Werewolf Horror', 'Zombie Horror', 
        'Splatter Horror', 'Found Footage Horror', 'Folk Horror', 
        'Giallo', 'Kaiju', 'Witch Horror', 'Body Horror'
    ],
    "Thriller": [
        'Thriller', 'Conspiracy Thriller', 'Political Thriller', 'Erotic Thriller', 
        'Psychological Thriller', 'Serial Killer', 'Heist', 'Crime', 'Cop Drama', 
        'Drug Crime', 'Gangster', 'Police Procedural', 'Legal Thriller', 
        'Suspense Mystery'
    ],
    "Science-Fiction": [
        'Sci-Fi', 'Sci-Fi Epic', 'Dystopian Sci-Fi', 'Cyber Thriller', 
        'Cyberpunk', 'Steampunk', 'Alien Invasion', 'Artificial Intelligence', 
        'Time Travel', 'Mecha', 'Superhero'
    ],
    "Fantasy": [
        'Fantasy', 'Dark Fantasy', 'Fantasy Epic', 'Sword & Sorcery', 
        'Supernatural Fantasy', 'Fairy Tale', 'Isekai', 'Slice of Life', 
        'Teen Fantasy'
    ],
    "Komödie": [
        'Comedy', 'Dark Comedy', 'Farce', 'High-Concept Comedy', 'Quirky Comedy', 
        'Raunchy Comedy', 'Romantic Comedy', 'Screwball Comedy', 'Slapstick', 
        'Stoner Comedy', 'Teen Comedy', 'Buddy Comedy', 'Parody', 'Satire', 
        'Sketch Comedy', 'Mockumentary', 'Sitcom'
    ],
    "Drama": [
        'Drama', 'Costume Drama', 'Cop Drama', 'Crime', 'Docudrama', 'Family', 
        'Historical Epic', 'Legal Drama', 'Medical Drama', 'Period Drama', 
        'Political Drama', 'Prison Drama', 'Psychological Drama', 'Showbiz Drama', 
        'Teen Drama', 'Tragedy', 'Workplace Drama', 'Financial Drama', 
        'Korean Drama', 'Biography', 'Dark Romance'
    ],
    "Romantik": [
        'Romance', 'Feel-Good Romance', 'Holiday Romance', 'Teen Romance', 
        'Tragic Romance', 'Steamy Romance', 'Romantic Epic'
    ],
    "Musik und Tanz": [
        'Musical', 'Classic Musical', 'Pop Musical', 'Rock Musical', 
        'Concert', 'Music', 'Jukebox Musical', 'Music Documentary'
    ],
    "Sport": [
        'Baseball', 'Basketball', 'Boxing', 'Extreme Sport', 'Football', 
        'Motorsport', 'Soccer', 'Sport', 'Sports Documentary', 'Water Sport'
    ],
    "Krimi": [
        'True Crime', 'Heist', 'Drug Crime', 'Gangster', 'Hard-boiled Detective', 
        'Bumbling Detective', 'Cozy Mystery'
    ],
    "Dokumentationen": [
        'Documentary', 'Crime Documentary', 'Food Documentary', 
        'Military Documentary', 'Nature Documentary', 
        'Political Documentary', 'Science & Technology Documentary', 
        'Travel Documentary', 'Faith & Spirituality Documentary', 'History Documentary', 
        'Game Show', 'News', 'Reality TV', 'Talk Show'
    ],
    "Western": [
        'Western', 'Contemporary Western', 'Classical Western', 
        'Spaghetti Western', 'Western Epic'
    ],
    "Animation": [
        'Animation', 'Anime', 'Hand-Drawn Animation', 'Stop Motion Animation', 
        'Adult Animation', 'Computer Animation'
    ],
    "Kinder und Familie": [
        'Family', 'Holiday Family', 'Teen Adventure', 'Coming-of-Age', 
        'Holiday', 'Holiday Animation', 'Holiday Comedy'
    ],
    "Historisch": [
        'Historical Epic', 'History', 'Period Drama'
    ],
    "Krieg": [
        'War', 'War Epic'
    ],
    "Mystery": [
        'Mystery', 'Whodunnit'
    ],
    "Sonstiges": [
        'Epic', 'Stand-Up', 'Iyashikei', 'Josei', 'Seinen', 'Shōjo', 
        'Shōnen', 'Soap Opera'
    ]
}



In [214]:
# Funktion zur Zuordnung von Genres zu mehreren Hauptkategorien
def assign_main_genres(genre_list, genre_categories):
    main_genres = []  # Liste für alle Hauptkategorien des Films
    # Für jedes Genre im Film prüfen, ob es in einer der Hauptkategorien vorhanden ist
    for category, subgenres in genre_categories.items():
        if any(genre in subgenres for genre in genre_list):
            main_genres.append(category)  # Hinzufügen der passenden Kategorie zur Liste
    # Falls keine Hauptkategorie zugeordnet wurde, in 'Sonstiges' hinzufügen
    if not main_genres:
        main_genres.append('Sonstiges')
    return main_genres

# Anwenden der Funktion auf jedes Genre
final_df['main_genres'] = final_df['genres'].apply(lambda x: assign_main_genres(ast.literal_eval(x), genre_categories))

# Überprüfen der neuen Spalte
print(final_df[['genres', 'main_genres']].head())

                                              genres  \
0                ['Survival', 'Adventure', 'Family']   
1  ['Dark Comedy', 'Psychological Horror', 'Psych...   
2  ['Farce', 'Holiday Comedy', 'Holiday Romance',...   
3  ['Adventure Epic', 'Epic', 'Historical Epic', ...   
4                        ['Caper', 'Crime', 'Drama']   

                                         main_genres  
0             [Abenteuer, Drama, Kinder und Familie]  
1               [Horror, Thriller, Komödie, Mystery]  
2     [Komödie, Drama, Romantik, Kinder und Familie]  
3  [Action, Abenteuer, Drama, Historisch, Krieg, ...  
4                                  [Thriller, Drama]  


In [215]:
#drop die alte genre spalte
final_df = final_df.drop('genres', axis=1)
final_df.columns

Index(['Title', 'Movie Link', 'Year', 'Duration', 'Rating', 'Votes',
       'Description', 'budget', 'grossWorldWide', 'gross_US_Canada',
       'opening_weekend_Gross', 'directors', 'writers', 'stars',
       'countries_origin', 'filming_locations', 'production_companies',
       'Languages', 'wins', 'nominations', 'oscars', 'release_date', 'id',
       'MPA_category', 'inflation_gross_opening', 'budget_inflation',
       'has_oscars', 'decade', 'main_genres'],
      dtype='object')

In [219]:
# Titel anpassen, keine Nummern drin
def extract_title(title):
    stripped_title = title.split('. ')[1]
    return stripped_title

final_df['Title'] = final_df['Title'].apply(extract_title)


In [220]:
final_df['Title']

0       Swiss Family Robinson
1                      Psycho
2               The Apartment
3                   Spartacus
4                  Breathless
                ...          
6595                   Cuckoo
6596            Back to Black
6597         Someone Like You
6598            Devara Part 1
6599        The Killer's Game
Name: Title, Length: 6598, dtype: object

In [222]:
final_df

,Title,Movie Link,Year,Duration,Rating,Votes,Description,budget,grossWorldWide,gross_US_Canada,...,nominations,oscars,release_date,id,MPA_category,inflation_gross_opening,budget_inflation,has_oscars,decade,main_genres
0,Swiss Family Robinson,https://www.imdb.com/title/tt0054357/?ref_=sr_t_1,1960,126,7.1,19000.0,A Swiss family must survive being shipwrecked ...,5000000.0,40357287.0,40356000.0,...,0,0,1960,tt0054357,All Ages (+0),1.472207e+05,5.279188e+07,0,60s,"[Abenteuer, Drama, Kinder und Familie]"
1,Psycho,https://www.imdb.com/title/tt0054215/?ref_=sr_t_2,1960,109,8.5,745000.0,A secretary on the run for embezzlement takes ...,806947.0,32248065.0,32181230.0,...,14,4,1960,tt0054215,Mature Audiences (+18),1.472207e+05,8.520050e+06,1,60s,"[Horror, Thriller, Komödie, Mystery]"
2,The Apartment,https://www.imdb.com/title/tt0053604/?ref_=sr_t_3,1960,125,8.3,206000.0,A Manhattan insurance clerk tries to rise in h...,3000000.0,18783481.0,18600000.0,...,8,5,1960,tt0053604,All Ages (+0),1.472207e+05,3.167513e+07,1,60s,"[Komödie, Drama, Romantik, Kinder und Familie]"
3,Spartacus,https://www.imdb.com/title/tt0054331/?ref_=sr_t_4,1960,197,7.9,147000.0,The slave Spartacus survives brutal training a...,12000000.0,1846975.0,1830650.0,...,11,4,1960,tt0054331,Parental Guidance (+13),9.730810e+05,1.267005e+08,1,60s,"[Action, Abenteuer, Drama, Historisch, Krieg, ..."
4,Breathless,https://www.imdb.com/title/tt0053472/?ref_=sr_t_5,1960,90,7.7,90000.0,"A small-time crook, hunted by the authorities ...",400000.0,594039.0,414173.0,...,4,0,1960,tt0053472,Not Rated/Other,3.423448e+05,4.223350e+06,0,60s,"[Thriller, Drama]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,Cuckoo,https://www.imdb.com/title/tt12349832/?ref_=sr...,2024,102,5.7,23000.0,A 17-year-old girl is forced to move with her ...,7000000.0,6680396.0,6217733.0,...,11,0,2024,tt12349832,Mature Audiences (+18),3.030437e+06,7.000000e+06,0,2020s,"[Horror, Thriller, Mystery]"
6596,Back to Black,https://www.imdb.com/title/tt21261712/?ref_=sr...,2024,122,6.3,25000.0,"The life and music of Amy Winehouse, through t...",30000000.0,51026615.0,6178165.0,...,7,0,2024,tt21261712,Mature Audiences (+18),2.835720e+06,3.000000e+07,0,2020s,"[Drama, Musik und Tanz]"
6597,Someone Like You,https://www.imdb.com/title/tt22334258/?ref_=sr...,2024,118,5.7,1800.0,"After the tragic loss of his best friend, a gr...",50000000.0,5937306.0,5937306.0,...,0,0,2024,tt22334258,Parental Guidance (+13),2.979692e+06,5.000000e+07,0,2020s,[Romantik]
6598,Devara Part 1,https://www.imdb.com/title/tt11821912/?ref_=sr...,2024,176,6.0,20000.0,A mighty sea warrior takes a violent stand aga...,36500000.0,7361414.0,5600000.0,...,0,0,2024,tt11821912,Unknown,5.600000e+06,3.650000e+07,0,2020s,"[Action, Abenteuer, Thriller, Drama, Sonstiges]"


In [221]:
# df als csv abspeichern
final_df.to_csv(os.path.join('..', 'Datasets', 'df_new_movies_cleaned.csv'))